# **AI for Regulatory Compliance** 
<h1 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   AI for Regulatory Compliance
</h1>

Note: Set up environment variable: Use .env file to put all AWS related key

# **Importing Libraries** 
<h1 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Importing Libraries
</h1>

In [2]:
!pip install -r requirements.txt

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------------------ --------------------- 1.3/2.9 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 7.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   -- ------------------------------------- 1.0/14.6 MB 8.4 MB/s eta 0:00:02
   ------ --------------------------------- 2.4/14.6 MB 5.2 MB/s eta 0:00:03
   -------- ------------------------------- 3.1/14.6 MB 5.8 MB/s eta 0:00:02
   -------- ------------------------------- 3.1/14.6 MB 5.8 MB/s eta 0:00:02
   ----------- ---------------------------- 4.2/14.6 MB 3.9 MB/s eta 0:00:03
   ----------- ---------------------------- 4.2/14.6 MB 3.9 MB/s eta 0:00:03
   ----------------- ---------------------- 6.3/14.6 MB 4.1 MB/s eta 0:00:03
   ----------------- ---------------------- 6.3/14.6 MB 4.1 MB/s eta 0:00:03
   -------------------- ------------------- 7.3/14.6 MB 4.0 MB/s eta 0:00:02
   -------------

In [2]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)


In [20]:
!pip install utils

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13936 sha256=4436cd4e9dcfaabcd2e2593c936d52e8ebdce43deabf859d204a611cfe88dcb9
  Stored in directory: c:\users\hp probook\appdata\local\pip\cache\wheels\b6\a1\81\1036477786ae0e17b522f6f5a838f9bc4288d1016fc5d0e1ec
Successfully built utils


In [1]:
from dotenv import load_dotenv
import os
import pymupdf
from tqdm import tqdm
from IPython import display, HTML
import logging
from botocore.exceptions import ClientError
import pandas as pd
import re
import ast
import boto3
import json
import numpy as np

logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)


# **Setting up environment variables**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Setting up environment variables
</h2>

In [204]:
# Load the .env file
load_dotenv()

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_default_region = os.getenv('AWS_DEFAULT_REGION')
# aws_default_region = 'ap-southeast-2'
# aws_default_region = 'ap-northeast-2'
aws_default_region = 'ap-northeast-1'
# aws_default_region = 'ap-south-1'

# DELETE output before push to github
# print(f"AWS Access Key: {aws_access_key_id}")
# print(f"AWS Secret Key: {aws_secret_access_key}")
print(f"AWS Region: {aws_default_region}")

AWS Region: ap-northeast-1


# Flow:
### 1. Data Extraction
### 2. Model Selection
### 3. Prompt Engineering
### 4. API Invocation
### 5. Parameter Tuning
### 6. Output Formatting

# **Data Extraction**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Data Extraction
</h2>

In [3]:
def extract_text_from_doc(filepath):
    doc = pymupdf.open(filepath)
    num_pages = len(doc)

    text_compilation = ""

    for page_num in tqdm(range(num_pages), desc="Processing PDF pages"):

        page = doc[page_num]
        text = page.get_text()

        text_compilation += "Page " + str(page_num) + ": " + text + "\n\n"

    return text_compilation

# **Model Selection**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Model Selection
</h2>

# - Claude 3 Haiku
# - Amazon Titan Premier

# **BNM Policy Changes**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   BNM Policy Changes
</h2>

In [5]:
def prompt_engineering(bnm_old_policy,bnm_new_policy):

    doc1_text = extract_text_from_doc(bnm_old_policy)
    
    doc2_text = extract_text_from_doc(bnm_new_policy)
    
    print('Document 1:\n' + doc1_text)
    print()
    print('Document 2:\n' + doc2_text)
    
    prompt = f"""
    You will be provided with Document 1 and Document 2 texts delimited by <>.
    Your task is to perform the following actions:
    1. Compare Document 1 and Document 2.
    2. Detect additions, modifications, or deletions.
    3. Focus on identifying changes in semantic meaning.
    4. Answer in JSON format with a consistent style as shown in the example provided below:
    [
    {{"Num":1, 
    "Affected Section/Paragraph": "Section 3.1", 
    "Summary of Impact": "The following additions have been made in the updated version of the document:
    1. Compliance History: A new criterion has been added to review any past legal issues or lawsuits related to the entity.
    2. Reputation: A new section has been introduced to assess the entity’s reputation in the industry and any past issues that could affect the partnership.
    Both of these points were not present in the original version"}},
    {{"Num":2, 
    "Affected Section/Paragraph": "Paragraph 3.3", 
    "Summary of Impact": "A new section, 'Incident Reporting and Escalation,' is added."}},
    {{"Num":3, 
    "Affected Section/Paragraph": "Based on old policy document: 3.4 Training and Awareness\nBased on new policy document: 3.5 Training and Awareness", 
    "Summary of Impact": "The section number has been moved from 3.4 to 3.5 due to the addition of a new section (3.3). The training and awareness program has been changed to optional."}}]
    5. Must include numbering of the section or paragraph inside the value of key 'Affected Section/Paragraph'.
    
    
    Document 1:
    <{doc1_text}>
    
    Document 2:
    <{doc2_text}>
    
    """
    return prompt

In [4]:
def invoke_claude_3_multimodal(prompt,model_id,temperature,top_p,top_k,file):
    """
    Invoke the Claude-3 multimodal model from Anthropic using AWS Bedrock runtime.

    Args:
        prompt (str): The text prompt to provide to the model.

    Returns:
        dict: The model's response dict.

    Raises:
        ValueError: If an invalid model id is provided.
    """
    # Initialize the Amazon Bedrock runtime client
    client = boto3.client(service_name="bedrock-runtime",
                    aws_access_key_id=aws_access_key_id,
                    aws_secret_access_key=aws_secret_access_key,
                    region_name=aws_default_region)

    # Prepare the prompt message
    message_content = []

    # Add text to the message content
    message_content.append({"type": "text", "text": prompt})

    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 2048,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "messages": [
            {
                "role": "user",
                "content": message_content,
            }
        ],
    }

    try:
        response = client.invoke_model(
            modelId=model_id,
            body=json.dumps(request_body),
        )

        result = json.loads(response.get("body").read())
        ans = result['content'][0]['text']
        json_data = re.search(r'\[.*\]', ans, re.DOTALL)
        ans= json_data.group()
        response['body'] = ans

        with open(file, "w") as file:
            # Write response to the file
            file.write(str(response))
        
        return response

    except ClientError as err:
        logger.error(
            "Couldn't invoke Claude 3 %s model. Here's why: %s: %s",
            model_id.split('.')[-1].capitalize(),
            err.response["Error"]["Code"],
            err.response["Error"]["Message"],
        )
        raise

In [68]:
def invoke_amazon_titan_model(prompt,model_id,temperature,top_p,file):
   
    # Initialize the Amazon Bedrock runtime client
    client = boto3.client(service_name="bedrock-runtime",
                    aws_access_key_id=aws_access_key_id,
                    aws_secret_access_key=aws_secret_access_key,
                    region_name=aws_default_region)


    request_body = {
        "inputText": prompt,
        "textGenerationConfig":{
            "maxTokenCount": 3072,
            "stopSequences" : [],
            "temperature": temperature,
            "topP": top_p,
        }
    }

    try:
        response = client.invoke_model(
            modelId=model_id,
            body=json.dumps(request_body),
        )

        result = json.loads(response.get("body").read())
        ans = result['results'][0]['outputText']
        json_data = re.search(r'\[.*\]', ans, re.DOTALL)
        ans= json_data.group()
        response['body'] = ans
        
        with open(file, "w") as file:
            # Write response to the file
            file.write(str(response))
        
        return response

    except ClientError as err:
        logger.error(
            "Couldn't invoke titan %s model. Here's why: %s: %s",
            model_id.split('.')[-1].capitalize(),
            err.response["Error"]["Code"],
            err.response["Error"]["Message"],
        )
        raise

In [6]:
price_per_1000_token={'Region':
           {
               'Asia Pacific (Sydney)':
                {'Anthropic':
                     {'Claude 3 Sonnet':
                          {'input':0.003,
                           'output':0.015
                          },
                      'Claude 3 Haiku':
                          {'input':0.00025,
                           'output':0.00125
                          }
                     }
                }
           ,
               'US East (N. Virginia)':
                    {'Amazon':
                         {'Amazon Titan Text Premier':
                              {'input':0.0005,
                               'output':0.0015
                              }
                         }
                    }           
          }
                     }   

In [15]:
def show_output_claude(file,model_name):
    # Open the file in read mode
    with open(file, "r") as f:
        # Read the entire file content
        content = f.read()
        
    # Convert string to a dictionary
    response_dict = ast.literal_eval(content)
    input_token_count = response_dict['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-input-token-count']
    output_token_count = response_dict['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-output-token-count']
    per_1000_input = price_per_1000_token['Region']['Asia Pacific (Sydney)']['Anthropic'][model_name]['input']
    per_1000_output = price_per_1000_token['Region']['Asia Pacific (Sydney)']['Anthropic'][model_name]['output']
    price_input = int(input_token_count)/1000*per_1000_input
    price_output = int(output_token_count)/1000*per_1000_output
    total_price = price_input+price_output

    # body_content = response_dict.get('body')
    body_content = response_dict['body']

    # Step 2: Convert JSON string to Python list (if the body is a JSON string)
    json_data = json.loads(body_content)
    
    # Step 3: Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(json_data)

   
    
    # Print the extracted values
    print("Summary:")
    print(f"Input Token Count: {input_token_count}")
    print(f"Output Token Count: {output_token_count}")
    print(f"Input Token Price: ${price_input}")
    print(f"Output Token Price: ${price_output}")
    print(f"Total Price for this run: ${total_price}")
    return df
    

In [8]:
def show_output_titan(file,model_name):
    # Open the file in read mode
    with open(file, "r") as f:
        # Read the entire file content
        content = f.read()
        
    # Convert string to a dictionary
    response_dict = ast.literal_eval(content)
    input_token_count = response_dict['inputTextTokenCount']
    output_token_count = response_dict['results'][0]['tokenCount']
    per_1000_input = price_per_1000_token['Region']['US East (N. Virginia)']['Amazon'][model_name]['input']
    per_1000_output = price_per_1000_token['Region']['US East (N. Virginia)']['Amazon'][model_name]['output']
    price_input = int(input_token_count)/1000*per_1000_input
    price_output = int(output_token_count)/1000*per_1000_output
    total_price = price_input+price_output

    body_content = response_dict['results'][0]['outputText']

    # Step 2: Convert JSON string to Python list (if the body is a JSON string)
    json_data = json.loads(body_content)

    # Step 3: Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(json_data)

   
    
    # Print the extracted values
    print("Summary:")
    print(f"Input Token Count: {input_token_count}")
    print(f"Output Token Count: {output_token_count}")
    print(f"Input Token Price: ${price_input}")
    print(f"Output Token Price: ${price_output}")
    print(f"Total Price for this run: ${total_price}")
    return df
    

# **Identifying eKYC Changes**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Identifying eKYC changes
</h2>

## **Amazon Titan Premier**
<h3 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Amazon Titan Premier
</h3>

In [201]:
output_file = "titan_premier_1.txt"
model_name= "Amazon Titan Text Premier"
model_id="amazon.titan-text-premier-v1:0"

In [35]:
print(prompt)


    You will be provided with Document 1 and Document 2 texts delimited by <>.
    Your task is to perform the following actions:
    1. Compare Document 1 and Document 2.
    2. Detect additions, modifications, or deletions.
    3. Focus on identifying changes in semantic meaning.
    4. Answer in JSON format with a consistent style as shown in the example provided below:
    [
    {"Num":1, 
    "Affected Section/Paragraph": "Section 3.1", 
    "Summary of Impact": "The following additions have been made in the updated version of the document:
    1. Compliance History: A new criterion has been added to review any past legal issues or lawsuits related to the entity.
    2. Reputation: A new section has been introduced to assess the entity’s reputation in the industry and any past issues that could affect the partnership.
    Both of these points were not present in the original version"},
    {"Num":2, 
    "Affected Section/Paragraph": "Paragraph 3.3", 
    "Summary of Impact": "A 

In [46]:
response = invoke_amazon_titan_model(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                     file=output_file)

In [202]:
output = show_output_titan(file=output_file,model_name=model_name)

Summary:
Input Token Count: 29649
Output Token Count: 606
Input Token Price: $0.014824500000000001
Output Token Price: $0.000909
Total Price for this run: $0.0157335


In [203]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output.to_html()))


,Num,Affected_Section,Summary_of_Impact,Affected_Paragraph
0,1,Paragraph 8.16,The requirement to submit information to the Bank for first-time implementation of e-KYC for legal persons has been removed.,"S 8.16 In respect of paragraph 8.13 (ii), a licensed person or a prescribed development financial institution shall give due regard to situations where there is potential for higher risk of misidentification and establish internal safeguard measures to address this risk."
1,2,Paragraph 8.16,The requirement to submit information to the Bank for first-time implementation of e-KYC for higher risk products without a credit transfer safeguard has been removed.,"S 8.16 For avoidance of doubt, this requirement also applies to a financial institution implementing e-KYC in the following situations for the first time: (i) e-KYC for legal persons; and/or (ii) e-KYC for higher risk products without a credit transfer safeguard."
2,3,Paragraph 10.1,The requirement to submit information to the Bank for first-time implementation of e-KYC for legal persons has been removed.,"S 10.1 Subject to paragraphs 8.1 and 8.3, where a licensed person or a prescribed development financial institution meets the requirements stipulated in this policy document and intends to implement an e-KYC solution described in paragraph 8.19 for the first time or change the appointed technology provider for the e-KYC solution, a complete list of information as set out in Appendix 5 shall be submitted to the Bank."
3,4,Paragraph 10.1,The requirement to submit information to the Bank for first-time implementation of e-KYC for higher risk products without a credit transfer safeguard has been removed.,"S 10.1 For avoidance of doubt, this requirement also applies to a financial institution implementing e-KYC in the following situations for the first time: (i) e-KYC for legal persons; and/or (ii) e-KYC for higher risk products without a credit transfer safeguard."


## **Claude 3 Haiku**
<h3 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Claude 3 Haiku
</h3>

In [34]:
output_file = "haiku_1.txt"
model_name= "Claude 3 Haiku"
model_id="anthropic.claude-3-haiku-20240307-v1:0"
prompt = prompt_engineering(r'..\samples\regulatory_policy_documents\ekyc_2020_06.pdf', r'..\samples\regulatory_policy_documents\ekyc_2024_04.pdf')


Processing PDF pages: 100%|███████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 330.86it/s]

Document 1:
Page 0: Issued on: 30 June 2020  
 
BNM/RH/PD 030-10 
       
 
 
 
 
 
 
 
 
 
 
 
 
Electronic Know-Your-Customer (e-KYC)  
 
 
 
 
 
 
 
 
 
 
 
Applicable to- 
1. Licensed banks  
2. Licensed investment banks 
3. Licensed Islamic banks 
4. Licensed life insurers 
5. Licensed family takaful operators 
6. Prescribed development financial institutions 
7. Licensed money-changing operators 
8. Licensed remittance service providers  
9. Approved non-bank issuers of designated payment instruments and designated Islamic payment 
instruments 


Page 1: Electronic Know-Your-Customer (e-KYC)  
 
 
 
 
Issued on: 30 June 2020  
 
 
 
TABLE OF CONTENTS 
 
Part A 
Overview ................................................................................................. 1 
1  
Introduction ........................................................................................ 1 
2  
Applicability .......................................................................................

In [136]:
response = invoke_claude_3_multimodal(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                      top_k=90,
                                     file=output_file)

In [140]:
output_1 = show_output_claude(file=output_file,model_name=model_name)

Summary:
Input Token Count: 29793
Output Token Count: 584
Input Token Price: $0.00744825
Output Token Price: $0.00073
Total Price for this run: $0.00817825


In [141]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output_1.to_html()))

,Num,Affected Section/Paragraph,Summary of Impact
0,1,"Sections 1, 2, 3, 4, 5, 6, 7","The updated policy document has expanded the scope to cover e-KYC implementation for both individuals and legal persons. The policy requirements have been enhanced to provide more detailed guidance on e-KYC solutions, including requirements on the use of artificial intelligence, machine learning or other forms of predictive algorithms, reliance on human representatives, and additional safeguards for higher risk financial products."
1,2,"Section 8.22, Appendix 3",New requirements have been introduced for financial institutions to ensure the technology provider appointed to provide the e-KYC solution conducts an external independent assessment on the e-KYC solution in accordance with the minimum scope and criteria specified. This is to identify the overall effectiveness and robustness of the e-KYC solution in detecting and mitigating ML/TF/PF and fraud risks.
2,3,"Section 8.23, 8.24, 8.25","New requirements have been introduced for financial institutions to perform due diligence on the identified technology provider and the e-KYC solution prior to implementation, and to conduct an independent assessment on the financial institution's own processes, procedures and controls prior to first-time implementation of an e-KYC solution. These requirements aim to ensure the overall effectiveness of e-KYC implementation."
3,4,Appendix 2,"More detailed requirements have been introduced on the measurement and assessment of False Acceptance Rate (FAR), including the adoption of a tiered approach for FAR threshold monitoring, review and notification to the Bank. This aims to strengthen the monitoring and management of e-KYC solution performance."
4,5,Appendix 4,"New requirements have been introduced for financial institutions to implement additional safeguards when offering higher risk financial products through e-KYC, such as current accounts, savings accounts and unrestricted investment accounts. This includes measures like requiring credit transfers from existing bank accounts and implementing ringfencing parameters to mitigate risks."


# **Identifying RMiT Changes**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Identifying RMiT Changes
</h2>

## **Amazon Titan Premier**
<h3 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Amazon Titan Premier
</h3>

In [77]:
output_file = "titan_premier_2.txt"
model_name= "Amazon Titan Text Premier"
model_id="amazon.titan-text-premier-v1:0"
prompt = prompt_engineering(r'..\samples\regulatory_policy_documents\rmit_2020_06.pdf', r'..\samples\regulatory_policy_documents\rmit_2023_06.pdf')

Processing PDF pages: 100%|███████████████████████████████████████████████████████████| 67/67 [00:00<00:00, 404.85it/s]

Document 1:
Page 0:   
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Risk Management in Technology (RMiT) 
 
 
 
 
 
 
 
 
Applicable to: 
1. Licensed banks 
2. Licensed investment banks 
3. Licensed Islamic banks 
4. Licensed insurers including professional reinsurers 
5. Licensed takaful operators including professional retakaful operators 
6. Prescribed development financial institutions 
7. Approved issuer of electronic money 
8. Operator of a designated payment system 
 
 
 
 
Issued on: 19 June 2020   
 
 
 
 
                    BNM/RH/PD 028-98 
 


Page 1: Risk Management in Technology                  
     2 of 50 
 
Issued on: 19 June 2020 
 
TABLE OF CONTENTS 
PART A OVERVIEW ............................................................................................... 3 
1 
Introduction ....................................................................................... 3 
2 
Applicability ....................................................................................... 3 
3 
Leg

In [78]:
response = invoke_amazon_titan_model(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                     file=output_file)

Couldn't invoke titan Titan-text-premier-v1:0 model. Here's why: ValidationException: Malformed input request: #/inputText: expected maxLength: 150000, actual: 298389, please reformat your input and try again.


ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #/inputText: expected maxLength: 150000, actual: 298389, please reformat your input and try again.

In [76]:
output = show_output_titan(file=output_file,model_name=model_name)

KeyError: 'inputTextTokenCount'

## **Claude 3 Haiku**
<h3 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Claude 3 Haiku
</h3>

In [142]:
output_file = "haiku_2.txt"
model_name= "Claude 3 Haiku"
model_id="anthropic.claude-3-haiku-20240307-v1:0"
prompt = prompt_engineering(r'..\samples\regulatory_policy_documents\rmit_2020_06.pdf', r'..\samples\regulatory_policy_documents\rmit_2023_06.pdf')

In [126]:
print(prompt)


    You will be provided with Document 1 and Document 2 texts delimited by <>.
    Your task is to perform the following actions:
    1. Compare Document 1 and Document 2.
    2. Detect additions, modifications, or deletions.
    3. Focus on identifying changes in semantic meaning.
    4. Answer in JSON format with a consistent style as shown in the example provided below:
    [
    {"Num":1, 
    "Affected Section/Paragraph": "Section 3.1", 
    "Summary of Impact": "The following additions have been made in the updated version of the document:
    1. Compliance History: A new criterion has been added to review any past legal issues or lawsuits related to the entity.
    2. Reputation: A new section has been introduced to assess the entity’s reputation in the industry and any past issues that could affect the partnership.
    Both of these points were not present in the original version"},
    {"Num":2, 
    "Affected Section/Paragraph": "Paragraph 3.3", 
    "Summary of Impact": "A 

In [136]:
response = invoke_claude_3_multimodal(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                      top_k=90,
                                     file=output_file)

In [143]:
output_2 = show_output_claude(file=output_file,model_name=model_name)

Summary:
Input Token Count: 71596
Output Token Count: 551
Input Token Price: $0.017899
Output Token Price: $0.00068875
Total Price for this run: $0.01858775


In [144]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output_2.to_html()))

,Num,Affected Section/Paragraph,Summary of Impact
0,1,Paragraph 1.3,The following additions have been made in the updated version of the document: 1. Inclusion of licensed digital banks and licensed Islamic digital banks in the list of financial institutions to which this policy document applies. 2. Inclusion of a new Appendix 10 which provides additional guidance to financial institutions for the assessment of common key risks and considerations of control measures when financial institutions adopt public cloud for critical systems.
1,2,Paragraph 4.1,"The effective date for this policy document has been updated. Paragraph 10.50, paragraph 15 and Appendix 10 come into effect on 1 June 2024 for financial institutions other than licensed digital banks and licensed Islamic digital banks, while the rest of the policy document comes into effect on 1 June 2023 for all financial institutions."
2,3,Paragraph 7.1,"The policy document on Risk Management in Technology (RMiT) issued on 1 January 2020 has been superseded, except for paragraphs 10.49, 10.50, 10.51 and 10.52 which shall remain applicable until 31 May 2024 in respect of financial institutions described in paragraph 4.1(a) and (b)."
3,4,Part C - Regulatory Process,"A new section on ""Consultation and Notification related to Cloud Services"" has been added, which requires financial institutions to consult the Bank prior to the first-time adoption of public cloud for critical systems, and to notify the Bank on any subsequent adoption of public cloud for critical systems."
4,5,Appendix 10,A new Appendix 10 has been added which provides additional guidance to financial institutions for the assessment of common key risks and considerations of control measures when financial institutions adopt public cloud for critical systems.


# **Impact Towards Current Policy**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Impact Towards Current Policy
</h2>

In [71]:
def prompt_engineering_2(internal_policy,output,text):

    doc1_text = extract_text_from_doc(internal_policy)
    
    print('Document 1:\n' + doc1_text)
    
    prompt = f"""
    You will be provided with Internal Policy texts delimited by <>.
    You will be provided with Changes in Central Bank of Malaysia {text} Policy texts delimited by <>.
    Your task is to perform the following actions:
    1. From Changes in Central Bank of Malaysia {text} Policy, identify how it affected section/paragraph of Internal Policy and its impact on Internal Policy.
    2. Answer in JSON format with a consistent style as shown in the example provided below:
    [
    {{"Num":1, 
    "Affected Section/Paragraph": "4.1", 
    "Summary of Impact": "The new policy (section 3.1) introduces several new criteria and adjustments compared to the current policy document(section 4.1). Here's a summary of the impacts:
    
    Compliance History (New in 3.1)
    Impact: The updated policy now requires a review of past legal issues or lawsuits, adding an extra layer of diligence which will necessitate additional legal checks and documentation.
    
    Reputation (New in 3.1)
    Impact: The updated policy mandates assessing the entity’s reputation in the industry and any past issues that could affect the partnership; compliance will require implementing a process for evaluating reputation, likely involving public records or third-party assessments."}},
    {{"Num":2, 
    "Affected Section/Paragraph": "4.4 Training and Awareness", 
    "Summary of Impact": "The company now has the option to decide whether to implement a training and awareness program or not."}}]
    3. Must include numbering of the section or paragraph inside the value of key 'Affected Section/Paragraph'.
    
    
    Internal Policy:
    <{doc1_text}>
    
    Changes in Central Bank of Malaysia {text} Policy:
    <{output}>

    
    """
    return prompt

## **eKYC**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   eKYC
</h2>

### **Impact of eKYC changes towards Access Management Procedure**
<h3 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Impact of eKYC changes towards Access Management Procedure
</h3>

In [146]:
prompt = prompt_engineering_2(r'..\samples\internal_policy_documents\Access_Management_Procedure_Sample.pdf',output_1.to_string(), 'eKYC')

Processing PDF pages: 100%|███████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 373.19it/s]

Document 1:
Page 0:  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Access Management Procedure 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Document ID 
AMP V1.0.1 
Effective Date 
2023-04-01 


Page 1:  
 
 
 
 
 
2 of 10 
 
 
Table of Contents 
1 
Purpose and Applicability....................................................................................................... 3 
2 
Related Policies, Guidelines, Procedures, and Applicable Laws ................................................. 3 
3 
Glossary of Key Terms............................................................................................................ 4 
4 
Roles and Responsibilities ...................................................................................................... 5 
5 
Access Management Overview............................................................................................... 6 
6 
Mandatory User Access Review.....................................................................................

In [145]:
output_1

,Num,Affected Section/Paragraph,Summary of Impact
0,1,"Sections 1, 2, 3, 4, 5, 6, 7",The updated policy document has expanded the s...
1,2,"Section 8.22, Appendix 3",New requirements have been introduced for fina...
2,3,"Section 8.23, 8.24, 8.25",New requirements have been introduced for fina...
3,4,Appendix 2,More detailed requirements have been introduce...
4,5,Appendix 4,New requirements have been introduced for fina...


In [147]:
prompt

'\n    You will be provided with Internal Policy texts delimited by <>.\n    You will be provided with Changes in Central Bank of Malaysia eKYC Policy texts delimited by <>.\n    Your task is to perform the following actions:\n    1. From Changes in Central Bank of Malaysia eKYC Policy, identify how it affected section/paragraph of Internal Policy and its impact on Internal Policy.\n    2. Answer in JSON format with a consistent style as shown in the example provided below:\n    [\n    {"Num":1, \n    "Affected Section/Paragraph": "4.1", \n    "Summary of Impact": "The new policy (section 3.1) introduces several new criteria and adjustments compared to the current policy document(section 4.1). Here\'s a summary of the impacts:\n    \n    Compliance History (New in 3.1)\n    Impact: The updated policy now requires a review of past legal issues or lawsuits, adding an extra layer of diligence which will necessitate additional legal checks and documentation.\n    \n    Reputation (New in 3

In [148]:
output_file = "haiku_1.1.txt"
model_name= "Claude 3 Haiku"
model_id="anthropic.claude-3-haiku-20240307-v1:0"
# aws_default_region = 'ap-southeast-2'
# aws_default_region = 'ap-northeast-2'
# aws_default_region = 'ap-northeast-1'
aws_default_region = 'ap-south-1'
# aws_default_region = 'us-east-1'

In [149]:
response = invoke_claude_3_multimodal(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                      top_k=90,
                                     file=output_file)

In [150]:
output = show_output_claude(file=output_file,model_name=model_name)

Summary:
Input Token Count: 3826
Output Token Count: 612
Input Token Price: $0.0009565
Output Token Price: $0.0007650000000000001
Total Price for this run: $0.0017215


In [151]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output.to_html()))

,Num,Affected Section/Paragraph,Summary of Impact
0,1,"4.1, 4.4","The new policy (section 3.1) introduces several new criteria and adjustments compared to the current policy document (section 4.1). The updated policy now requires a review of past legal issues or lawsuits, adding an extra layer of diligence which will necessitate additional legal checks and documentation. The company now has the option to decide whether to implement a training and awareness program or not."
1,2,"5.2, 5.3","The new policy requirements in sections 8.22, Appendix 3 mandate that financial institutions ensure the technology provider appointed to provide the e-KYC solution conducts an external independent assessment on the e-KYC solution. This is to identify the overall effectiveness and robustness of the e-KYC solution in detecting and mitigating ML/TF/PF and fraud risks, which will require additional due diligence and assessment processes."
2,3,"7.1, 7.2","The new requirements in sections 8.23, 8.24, 8.25 require financial institutions to perform due diligence on the identified technology provider and the e-KYC solution prior to implementation, and to conduct an independent assessment on the financial institution's own processes, procedures and controls prior to first-time implementation of an e-KYC solution. These additional requirements aim to ensure the overall effectiveness of e-KYC implementation."
3,4,6.1,"The more detailed requirements introduced in Appendix 2 on the measurement and assessment of False Acceptance Rate (FAR), including the adoption of a tiered approach for FAR threshold monitoring, review and notification to the Bank, will strengthen the monitoring and management of e-KYC solution performance."
4,5,7.4,"The new requirements in Appendix 4 for financial institutions to implement additional safeguards when offering higher risk financial products through e-KYC, such as requiring credit transfers from existing bank accounts and implementing ringfencing parameters, aim to mitigate the risks associated with these higher risk financial products."


### **Impact of eKYC changes towards Change Management Procedure**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Impact of eKYC changes towards Change Management Procedure
</h2>

In [152]:
prompt = prompt_engineering_2(r'..\samples\internal_policy_documents\Change_Management_Procedure_Sample.pdf',output_1.to_string(), 'eKYC')

Processing PDF pages: 100%|███████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 491.18it/s]

Document 1:
Page 0:  
 
INTERNAL 
 
 
 
1 of 11 
 
 
 
 
 
 
 
 
Change Management Procedure 
 
 
 
 
Document ID 
CMP.v.1.0.0 
Effective Date 
2023-04-01 


Page 1:  
 
INTERNAL 
 
 
 
2 of 11 
 
Table of Contents 
1. Purpose and Applicability .................................................................................................................. 3 
2. Related Policies and Procedures ........................................................................................................ 3 
3. Glossary of Key Terms ........................................................................................................................ 4 
4. Overview ............................................................................................................................................ 5 
5. Roles and Responsibilities .................................................................................................................. 6 
6. Categories of Changes and Handling ...

In [153]:
output_1

,Num,Affected Section/Paragraph,Summary of Impact
0,1,"Sections 1, 2, 3, 4, 5, 6, 7",The updated policy document has expanded the s...
1,2,"Section 8.22, Appendix 3",New requirements have been introduced for fina...
2,3,"Section 8.23, 8.24, 8.25",New requirements have been introduced for fina...
3,4,Appendix 2,More detailed requirements have been introduce...
4,5,Appendix 4,New requirements have been introduced for fina...


In [154]:
prompt

'\n    You will be provided with Internal Policy texts delimited by <>.\n    You will be provided with Changes in Central Bank of Malaysia eKYC Policy texts delimited by <>.\n    Your task is to perform the following actions:\n    1. From Changes in Central Bank of Malaysia eKYC Policy, identify how it affected section/paragraph of Internal Policy and its impact on Internal Policy.\n    2. Answer in JSON format with a consistent style as shown in the example provided below:\n    [\n    {"Num":1, \n    "Affected Section/Paragraph": "4.1", \n    "Summary of Impact": "The new policy (section 3.1) introduces several new criteria and adjustments compared to the current policy document(section 4.1). Here\'s a summary of the impacts:\n    \n    Compliance History (New in 3.1)\n    Impact: The updated policy now requires a review of past legal issues or lawsuits, adding an extra layer of diligence which will necessitate additional legal checks and documentation.\n    \n    Reputation (New in 3

In [155]:
output_file = "haiku_1.2.txt"
model_name= "Claude 3 Haiku"
model_id="anthropic.claude-3-haiku-20240307-v1:0"
# aws_default_region = 'ap-southeast-2'
# aws_default_region = 'ap-northeast-2'
# aws_default_region = 'ap-northeast-1'
aws_default_region = 'ap-south-1'
# aws_default_region = 'us-east-1'

In [156]:
response = invoke_claude_3_multimodal(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                      top_k=90,
                                     file=output_file)

In [157]:
output = show_output_claude(file=output_file,model_name=model_name)

Summary:
Input Token Count: 3663
Output Token Count: 856
Input Token Price: $0.00091575
Output Token Price: $0.00107
Total Price for this run: $0.00198575


In [158]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output.to_html()))

,Num,Affected Section/Paragraph,Summary of Impact
0,1,4.1,"The new policy (section 3.1) introduces several new criteria and adjustments compared to the current policy document(section 4.1). Here's a summary of the impacts:\n\nCompliance History (New in 3.1)\nImpact: The updated policy now requires a review of past legal issues or lawsuits, adding an extra layer of diligence which will necessitate additional legal checks and documentation.\n\nReputation (New in 3.1)\nImpact: The updated policy mandates assessing the entity's reputation in the industry and any past issues that could affect the partnership; compliance will require implementing a process for evaluating reputation, likely involving public records or third-party assessments."
1,2,4.4 Training and Awareness,The company now has the option to decide whether to implement a training and awareness program or not.
2,3,"Sections 1, 2, 3, 4, 5, 6, 7","The updated policy document has expanded the scope to cover e-KYC implementation for both individuals and legal persons. The policy requirements have been enhanced to provide more detailed guidance on e-KYC solutions, including requirements on the use of artificial intelligence, machine learning or other forms of predictive algorithms, reliance on human representatives, and additional safeguards for higher risk financial products."
3,4,"Section 8.22, Appendix 3",New requirements have been introduced for financial institutions to ensure the technology provider appointed to provide the e-KYC solution conducts an external independent assessment on the e-KYC solution in accordance with the minimum scope and criteria specified. This is to identify the overall effectiveness and robustness of the e-KYC solution in detecting and mitigating ML/TF/PF and fraud risks.
4,5,"Section 8.23, 8.24, 8.25","New requirements have been introduced for financial institutions to perform due diligence on the identified technology provider and the e-KYC solution prior to implementation, and to conduct an independent assessment on the financial institution's own processes, procedures and controls prior to first-time implementation of an e-KYC solution. These requirements aim to ensure the overall effectiveness of e-KYC implementation."
5,6,Appendix 2,"More detailed requirements have been introduced on the measurement and assessment of False Acceptance Rate (FAR), including the adoption of a tiered approach for FAR threshold monitoring, review and notification to the Bank. This aims to strengthen the monitoring and management of e-KYC solution performance."
6,7,Appendix 4,"New requirements have been introduced for financial institutions to implement additional safeguards when offering higher risk financial products through e-KYC, such as current accounts, savings accounts and unrestricted investment accounts. This includes measures like requiring credit transfers from existing bank accounts and implementing ringfencing parameters to mitigate risks."


### **Impact of eKYC changes towards Data Management Procedure**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Impact of eKYC changes towards Data Management Procedure
</h2>

In [159]:
prompt = prompt_engineering_2(r'..\samples\internal_policy_documents\Data_Management_Procedure_Sample.pdf',output_1.to_string(), 'eKYC')

Processing PDF pages: 100%|███████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 354.96it/s]

Document 1:
Page 0:  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Data Management Procedure 
 
 
 
Document ID 
DMP V1.0.1 
Effective Date 
2023-04-01 


Page 1:  
Page 2 of 10 
 
Tabe of Contents 
1 
Purpose and Applicability................................................................................... 
3 
2 
Related Policies, Guidelines, Procedures, and Applicable Laws......................... 
3 
3 
Glossary of Key Terms........................................................................................ 
4 
4 
Roles and Responsibilities.................................................................................. 
5 
5 
Data Management Overview.............................................................................. 
6 
6 
Data Lifecycle Management............................................................................... 
9 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


Page 2:  
Page 3 of 10 
 
1 Purpose and Applicability 
1.1 
The Data Management Procedure (DMP) est

In [160]:
output_1

,Num,Affected Section/Paragraph,Summary of Impact
0,1,"Sections 1, 2, 3, 4, 5, 6, 7",The updated policy document has expanded the s...
1,2,"Section 8.22, Appendix 3",New requirements have been introduced for fina...
2,3,"Section 8.23, 8.24, 8.25",New requirements have been introduced for fina...
3,4,Appendix 2,More detailed requirements have been introduce...
4,5,Appendix 4,New requirements have been introduced for fina...


In [161]:
prompt

'\n    You will be provided with Internal Policy texts delimited by <>.\n    You will be provided with Changes in Central Bank of Malaysia eKYC Policy texts delimited by <>.\n    Your task is to perform the following actions:\n    1. From Changes in Central Bank of Malaysia eKYC Policy, identify how it affected section/paragraph of Internal Policy and its impact on Internal Policy.\n    2. Answer in JSON format with a consistent style as shown in the example provided below:\n    [\n    {"Num":1, \n    "Affected Section/Paragraph": "4.1", \n    "Summary of Impact": "The new policy (section 3.1) introduces several new criteria and adjustments compared to the current policy document(section 4.1). Here\'s a summary of the impacts:\n    \n    Compliance History (New in 3.1)\n    Impact: The updated policy now requires a review of past legal issues or lawsuits, adding an extra layer of diligence which will necessitate additional legal checks and documentation.\n    \n    Reputation (New in 3

In [162]:
output_file = "haiku_1.3.txt"
model_name= "Claude 3 Haiku"
model_id="anthropic.claude-3-haiku-20240307-v1:0"
# aws_default_region = 'ap-southeast-2'
# aws_default_region = 'ap-northeast-2'
# aws_default_region = 'ap-northeast-1'
aws_default_region = 'ap-south-1'
# aws_default_region = 'us-east-1'

In [163]:
response = invoke_claude_3_multimodal(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                      top_k=90,
                                     file=output_file)

In [164]:
output = show_output_claude(file=output_file,model_name=model_name)

Summary:
Input Token Count: 4549
Output Token Count: 739
Input Token Price: $0.00113725
Output Token Price: $0.00092375
Total Price for this run: $0.002061


In [165]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output.to_html()))

,Num,Affected Section/Paragraph,Summary of Impact
0,1,"4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7","The new policy (sections 1, 2, 3, 4, 5, 6, 7) has expanded the scope to cover e-KYC implementation for both individuals and legal persons. The policy requirements have been enhanced to provide more detailed guidance on e-KYC solutions, including requirements on the use of artificial intelligence, machine learning or other forms of predictive algorithms, reliance on human representatives, and additional safeguards for higher risk financial products. This will require the organization to review and update its internal policies and procedures to align with the new regulatory requirements."
1,2,6.2 Data Storage,"New requirements have been introduced for financial institutions to ensure the technology provider appointed to provide the e-KYC solution conducts an external independent assessment on the e-KYC solution in accordance with the minimum scope and criteria specified (section 8.22, Appendix 3). This will require the organization to implement a process to verify the independent assessment of the e-KYC solution and its effectiveness in detecting and mitigating ML/TF/PF and fraud risks."
2,3,6.3 Data Usage,"New requirements have been introduced for financial institutions to perform due diligence on the identified technology provider and the e-KYC solution prior to implementation, and to conduct an independent assessment on the financial institution's own processes, procedures and controls prior to first-time implementation of an e-KYC solution (sections 8.23, 8.24, 8.25). This will require the organization to implement a comprehensive due diligence and assessment process to ensure the overall effectiveness of e-KYC implementation."
3,4,6.2 Data Storage,"More detailed requirements have been introduced on the measurement and assessment of False Acceptance Rate (FAR), including the adoption of a tiered approach for FAR threshold monitoring, review and notification to the Bank (Appendix 2). This will require the organization to enhance its monitoring and management of e-KYC solution performance to ensure compliance with the new regulatory requirements."
4,5,6.3 Data Usage,"New requirements have been introduced for financial institutions to implement additional safeguards when offering higher risk financial products through e-KYC, such as current accounts, savings accounts and unrestricted investment accounts (Appendix 4). This includes measures like requiring credit transfers from existing bank accounts and implementing ringfencing parameters to mitigate risks. The organization will need to review and update its processes and controls for higher risk financial products to align with the new regulatory requirements."


### **Impact of eKYC changes towards Incident Management Procedure**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Impact of eKYC changes towards Incident Management Procedure
</h2>

In [166]:
prompt = prompt_engineering_2(r'..\samples\internal_policy_documents\Incident_Management_Procedure_Sample.pdf',output_1.to_string(), 'eKYC')

Processing PDF pages: 100%|█████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 338.87it/s]

Document 1:
Page 0:  
 
 
 
 
 
 
 
 
 
 
 
 
Incident Management Procedure 
 
 
 
 
 
 
 
 
 
 
Document ID 
IMP.v.1.0.0 
Effective Date 
2023-04-01 
 
 
 


Page 1:  
 
 
 
 
 
2 of 9 
 
Table of Contents 
1. 
Purpose and Applicability ................................................................................. 3 
2. 
Related Policies and Procedures ....................................................................... 3 
3. 
Glossary of Key Terms...................................................................................... 4 
4. 
Roles and Responsibilities................................................................................ 5 
5. 
Incident Mangement Overview .......................................................................... 6 
6. 
Incident Categorisation and Assessment ............................................................ 8 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


Page 2:  
 
 
 
 
 
3 of 9 
 
 
 
1. Purpose and Applicability 
1.1 
The Inciden

In [167]:
output_1

,Num,Affected Section/Paragraph,Summary of Impact
0,1,"Sections 1, 2, 3, 4, 5, 6, 7",The updated policy document has expanded the s...
1,2,"Section 8.22, Appendix 3",New requirements have been introduced for fina...
2,3,"Section 8.23, 8.24, 8.25",New requirements have been introduced for fina...
3,4,Appendix 2,More detailed requirements have been introduce...
4,5,Appendix 4,New requirements have been introduced for fina...


In [168]:
prompt

'\n    You will be provided with Internal Policy texts delimited by <>.\n    You will be provided with Changes in Central Bank of Malaysia eKYC Policy texts delimited by <>.\n    Your task is to perform the following actions:\n    1. From Changes in Central Bank of Malaysia eKYC Policy, identify how it affected section/paragraph of Internal Policy and its impact on Internal Policy.\n    2. Answer in JSON format with a consistent style as shown in the example provided below:\n    [\n    {"Num":1, \n    "Affected Section/Paragraph": "4.1", \n    "Summary of Impact": "The new policy (section 3.1) introduces several new criteria and adjustments compared to the current policy document(section 4.1). Here\'s a summary of the impacts:\n    \n    Compliance History (New in 3.1)\n    Impact: The updated policy now requires a review of past legal issues or lawsuits, adding an extra layer of diligence which will necessitate additional legal checks and documentation.\n    \n    Reputation (New in 3

In [169]:
output_file = "haiku_1.4.txt"
model_name= "Claude 3 Haiku"
model_id="anthropic.claude-3-haiku-20240307-v1:0"
# aws_default_region = 'ap-southeast-2'
# aws_default_region = 'ap-northeast-2'
# aws_default_region = 'ap-northeast-1'
aws_default_region = 'ap-south-1'
# aws_default_region = 'us-east-1'

In [170]:
response = invoke_claude_3_multimodal(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                      top_k=90,
                                     file=output_file)

In [171]:
output = show_output_claude(file=output_file,model_name=model_name)

Summary:
Input Token Count: 2915
Output Token Count: 669
Input Token Price: $0.00072875
Output Token Price: $0.0008362500000000001
Total Price for this run: $0.001565


In [172]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output.to_html()))

,Num,Affected Section/Paragraph,Summary of Impact
0,1,4.1 Purpose and Applicability,"The new policy (sections 1, 2, 3, 4, 5, 6, 7) has expanded the scope to cover e-KYC implementation for both individuals and legal persons. The policy requirements have been enhanced to provide more detailed guidance on e-KYC solutions, including requirements on the use of artificial intelligence, machine learning or other forms of predictive algorithms, reliance on human representatives, and additional safeguards for higher risk financial products. This will require the institution to review and update its internal policy to align with the new regulatory requirements."
1,2,"Section 8.22, Appendix 3","The new policy has introduced requirements for financial institutions to ensure the technology provider appointed to provide the e-KYC solution conducts an external independent assessment on the e-KYC solution. This will impact the institution's vendor management and due diligence processes, as they will need to incorporate these new requirements when selecting and onboarding e-KYC technology providers."
2,3,"Section 8.23, 8.24, 8.25","The new policy has introduced requirements for financial institutions to perform due diligence on the identified technology provider and the e-KYC solution prior to implementation, and to conduct an independent assessment on the institution's own processes, procedures and controls prior to first-time implementation of an e-KYC solution. This will require the institution to establish new procedures and controls to comply with these requirements before deploying any e-KYC solution."
3,4,Appendix 2,"The new policy has introduced more detailed requirements on the measurement and assessment of False Acceptance Rate (FAR), including the adoption of a tiered approach for FAR threshold monitoring, review and notification to the Bank. This will impact the institution's e-KYC performance monitoring and reporting processes, as they will need to implement the new FAR monitoring and management requirements."
4,5,Appendix 4,"The new policy has introduced requirements for financial institutions to implement additional safeguards when offering higher risk financial products through e-KYC, such as current accounts, savings accounts and unrestricted investment accounts. This includes measures like requiring credit transfers from existing bank accounts and implementing ringfencing parameters. The institution will need to update its product offering and onboarding processes to comply with these new requirements."


## **RMiT**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   RMiT
</h2>

### **Impact of RMiT changes towards Access Management Procedure**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Impact of RMiT changes towards Access Management Procedure
</h2>

In [97]:
prompt = prompt_engineering_2(r'..\samples\internal_policy_documents\Access_Management_Procedure_Sample.pdf',output_2.to_string(), 'RMiT')

Processing PDF pages: 100%|███████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 328.52it/s]

Document 1:
Page 0:  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Access Management Procedure 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Document ID 
AMP V1.0.1 
Effective Date 
2023-04-01 


Page 1:  
 
 
 
 
 
2 of 10 
 
 
Table of Contents 
1 
Purpose and Applicability....................................................................................................... 3 
2 
Related Policies, Guidelines, Procedures, and Applicable Laws ................................................. 3 
3 
Glossary of Key Terms............................................................................................................ 4 
4 
Roles and Responsibilities ...................................................................................................... 5 
5 
Access Management Overview............................................................................................... 6 
6 
Mandatory User Access Review.....................................................................................

In [98]:
output_file = "haiku_2.1.txt"
model_name= "Claude 3 Haiku"
model_id="anthropic.claude-3-haiku-20240307-v1:0"
# aws_default_region = 'ap-southeast-2'
# aws_default_region = 'ap-northeast-2'
# aws_default_region = 'ap-northeast-1'
aws_default_region = 'ap-south-1'
# aws_default_region = 'us-east-1'

In [99]:
response = invoke_claude_3_multimodal(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                      top_k=90,
                                     file=output_file)

In [101]:
output = show_output_claude(file=output_file,model_name=model_name)

Summary:
Input Token Count: 3815
Output Token Count: 602
Input Token Price: $0.00095375
Output Token Price: $0.0007525
Total Price for this run: $0.0017062499999999999


In [102]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output.to_html()))

,Num,Affected Section/Paragraph,Summary of Impact
0,1,1.2,"The new policy (paragraph 1.3) expands the scope of the policy document to include licensed digital banks and licensed Islamic digital banks, in addition to the existing financial institutions. This will require the institution to review and potentially update its internal policy to ensure alignment with the expanded scope."
1,2,1.3,"The updated effective date (paragraph 4.1) for the new policy document will require the institution to review and potentially update the internal policy to ensure compliance with the new effective dates. Specifically, paragraphs 10.50, 15, and Appendix 10 come into effect on 1 June 2024 for financial institutions other than licensed digital banks and licensed Islamic digital banks, while the rest of the policy document comes into effect on 1 June 2023 for all financial institutions."
2,3,2.1(a),"The superseding of the previous RMiT policy document (paragraph 7.1) will require the institution to review and update the internal policy to reflect the changes in the new RMiT policy, except for paragraphs 10.49, 10.50, 10.51, and 10.52, which shall remain applicable until 31 May 2024 for the specified financial institutions."
3,4,2.1,"The addition of a new section on ""Consultation and Notification related to Cloud Services"" (Part C - Regulatory Process) will require the institution to review and potentially update its internal policy to ensure compliance with the new requirements for consulting the Bank prior to the first-time adoption of public cloud for critical systems, and notifying the Bank on any subsequent adoption of public cloud for critical systems."
4,5,2.1,The addition of a new Appendix 10 (Appendix 10) will require the institution to review and potentially update its internal policy to incorporate the guidance provided for the assessment of common key risks and considerations of control measures when adopting public cloud for critical systems.


### **Impact of RMiT changes towards Change Management Procedure**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Impact of RMiT changes towards Change Management Procedure
</h2>

In [103]:
prompt = prompt_engineering_2(r'..\samples\internal_policy_documents\Change_Management_Procedure_Sample.pdf',output_2.to_string(), 'RMiT')

Processing PDF pages: 100%|███████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 526.89it/s]

Document 1:
Page 0:  
 
INTERNAL 
 
 
 
1 of 11 
 
 
 
 
 
 
 
 
Change Management Procedure 
 
 
 
 
Document ID 
CMP.v.1.0.0 
Effective Date 
2023-04-01 


Page 1:  
 
INTERNAL 
 
 
 
2 of 11 
 
Table of Contents 
1. Purpose and Applicability .................................................................................................................. 3 
2. Related Policies and Procedures ........................................................................................................ 3 
3. Glossary of Key Terms ........................................................................................................................ 4 
4. Overview ............................................................................................................................................ 5 
5. Roles and Responsibilities .................................................................................................................. 6 
6. Categories of Changes and Handling ...

In [104]:
output_file = "haiku_2.2.txt"
model_name= "Claude 3 Haiku"
model_id="anthropic.claude-3-haiku-20240307-v1:0"
# aws_default_region = 'ap-southeast-2'
# aws_default_region = 'ap-northeast-2'
# aws_default_region = 'ap-northeast-1'
aws_default_region = 'ap-south-1'
# aws_default_region = 'us-east-1'

In [105]:
response = invoke_claude_3_multimodal(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                      top_k=90,
                                     file=output_file)

In [107]:
output = show_output_claude(file=output_file,model_name=model_name)

Summary:
Input Token Count: 3652
Output Token Count: 488
Input Token Price: $0.0009130000000000001
Output Token Price: $0.00061
Total Price for this run: $0.001523


In [108]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output.to_html()))

,Num,Affected Section/Paragraph,Summary of Impact
0,1,1.3,"The new policy (section 1.3) expands the scope of the policy document to include licensed digital banks and licensed Islamic digital banks, in addition to the existing financial institutions. This will require the organization to review and potentially update its internal policy to ensure compliance with the expanded scope."
1,2,4.1,The updated effective date for the policy document (section 4.1) will require the organization to review the timeline for implementing the changes in its internal policy and ensure compliance by the specified dates.
2,3,7.1,"The superseding of the previous RMiT policy document (section 7.1), except for certain paragraphs, will require the organization to review its internal policy and align it with the updated RMiT policy document."
3,4,Part C - Regulatory Process,"The addition of a new section on ""Consultation and Notification related to Cloud Services"" (Part C) will require the organization to establish a process for consulting with the Bank prior to the first-time adoption of public cloud for critical systems, and for notifying the Bank on any subsequent adoptions. This will need to be incorporated into the organization's internal policy and procedures."
4,5,Appendix 10,The addition of a new Appendix 10 providing guidance on the assessment of risks and control measures for the adoption of public cloud for critical systems will require the organization to review and potentially update its internal policy and procedures to align with the new guidance.


### **Impact of RMiT changes towards Data Management Procedure**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Impact of RMiT changes towards Data Management Procedure
</h2>

In [109]:
prompt = prompt_engineering_2(r'..\samples\internal_policy_documents\Data_Management_Procedure_Sample.pdf',output_2.to_string(), 'RMiT')

Processing PDF pages: 100%|███████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 361.27it/s]

Document 1:
Page 0:  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Data Management Procedure 
 
 
 
Document ID 
DMP V1.0.1 
Effective Date 
2023-04-01 


Page 1:  
Page 2 of 10 
 
Tabe of Contents 
1 
Purpose and Applicability................................................................................... 
3 
2 
Related Policies, Guidelines, Procedures, and Applicable Laws......................... 
3 
3 
Glossary of Key Terms........................................................................................ 
4 
4 
Roles and Responsibilities.................................................................................. 
5 
5 
Data Management Overview.............................................................................. 
6 
6 
Data Lifecycle Management............................................................................... 
9 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


Page 2:  
Page 3 of 10 
 
1 Purpose and Applicability 
1.1 
The Data Management Procedure (DMP) est

In [110]:
output_file = "haiku_2.3.txt"
model_name= "Claude 3 Haiku"
model_id="anthropic.claude-3-haiku-20240307-v1:0"
# aws_default_region = 'ap-southeast-2'
# aws_default_region = 'ap-northeast-2'
# aws_default_region = 'ap-northeast-1'
aws_default_region = 'ap-south-1'
# aws_default_region = 'us-east-1'

In [111]:
response = invoke_claude_3_multimodal(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                      top_k=90,
                                     file=output_file)

In [112]:
output = show_output_claude(file=output_file,model_name=model_name)

Summary:
Input Token Count: 4538
Output Token Count: 439
Input Token Price: $0.0011345
Output Token Price: $0.00054875
Total Price for this run: $0.00168325


In [113]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output.to_html()))

,Num,Affected Section/Paragraph,Summary of Impact
0,1,4.1,"The effective date for this policy document has been updated. Paragraph 10.50, paragraph 15 and Appendix 10 come into effect on 1 June 2024 for financial institutions other than licensed digital banks and licensed Islamic digital banks, while the rest of the policy document comes into effect on 1 June 2023 for all financial institutions."
1,2,7.1,"The policy document on Risk Management in Technology (RMiT) issued on 1 January 2020 has been superseded, except for paragraphs 10.49, 10.50, 10.51 and 10.52 which shall remain applicable until 31 May 2024 in respect of financial institutions described in paragraph 4.1(a) and (b)."
2,3,Part C - Regulatory Process,"A new section on ""Consultation and Notification related to Cloud Services"" has been added, which requires financial institutions to consult the Bank prior to the first-time adoption of public cloud for critical systems, and to notify the Bank on any subsequent adoption of public cloud for critical systems."
3,4,Appendix 10,A new Appendix 10 has been added which provides additional guidance to financial institutions for the assessment of common key risks and considerations of control measures when financial institutions adopt public cloud for critical systems.


### **Impact of RMiT changes towards Incident Management Procedure**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Impact of RMiT changes towards Incident Management Procedure
</h2>

In [114]:
prompt = prompt_engineering_2(r'..\samples\internal_policy_documents\Incident_Management_Procedure_Sample.pdf',output_2.to_string(), 'RMiT')

Processing PDF pages: 100%|█████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 396.05it/s]

Document 1:
Page 0:  
 
 
 
 
 
 
 
 
 
 
 
 
Incident Management Procedure 
 
 
 
 
 
 
 
 
 
 
Document ID 
IMP.v.1.0.0 
Effective Date 
2023-04-01 
 
 
 


Page 1:  
 
 
 
 
 
2 of 9 
 
Table of Contents 
1. 
Purpose and Applicability ................................................................................. 3 
2. 
Related Policies and Procedures ....................................................................... 3 
3. 
Glossary of Key Terms...................................................................................... 4 
4. 
Roles and Responsibilities................................................................................ 5 
5. 
Incident Mangement Overview .......................................................................... 6 
6. 
Incident Categorisation and Assessment ............................................................ 8 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


Page 2:  
 
 
 
 
 
3 of 9 
 
 
 
1. Purpose and Applicability 
1.1 
The Inciden

In [115]:
output_file = "haiku_2.4.txt"
model_name= "Claude 3 Haiku"
model_id="anthropic.claude-3-haiku-20240307-v1:0"
# aws_default_region = 'ap-southeast-2'
# aws_default_region = 'ap-northeast-2'
# aws_default_region = 'ap-northeast-1'
aws_default_region = 'ap-south-1'
# aws_default_region = 'us-east-1'

In [116]:
response = invoke_claude_3_multimodal(prompt,
                                      model_id=model_id,
                                      temperature=0.2,
                                      top_p=0.3,
                                      top_k=90,
                                     file=output_file)

In [118]:
output = show_output_claude(file=output_file,model_name=model_name)

Summary:
Input Token Count: 2904
Output Token Count: 660
Input Token Price: $0.000726
Output Token Price: $0.0008250000000000001
Total Price for this run: $0.001551


In [119]:
from IPython.display import display, HTML
# Convert DataFrame to HTML with a DataTable for interactive scrolling
display(HTML(output.to_html()))

,Num,Affected Section/Paragraph,Summary of Impact
0,1,1.2,The updated policy document now includes licensed digital banks and licensed Islamic digital banks in the list of financial institutions to which this policy document applies. This expansion of scope will require these newly included entities to review and align their incident management procedures with the requirements outlined in the updated policy.
1,2,1.3,"The effective date for the implementation of this policy document has been updated. Certain sections, such as paragraph 10.50, paragraph 15, and Appendix 10, will come into effect on 1 June 2024 for financial institutions other than licensed digital banks and licensed Islamic digital banks, while the rest of the policy document will come into effect on 1 June 2023 for all financial institutions. This staggered implementation timeline will require the organization to carefully plan and execute the necessary changes to align with the new effective dates."
2,3,2.1(a),"The previous policy document on Risk Management in Technology (RMiT) issued on 1 January 2020 has been superseded, except for paragraphs 10.49, 10.50, 10.51, and 10.52, which shall remain applicable until 31 May 2024 for the specified financial institutions. This change will require the organization to review and update its references to the RMiT policy document and ensure compliance with the new policy requirements."
3,4,Part C - Regulatory Process,"A new section on ""Consultation and Notification related to Cloud Services"" has been added, which requires financial institutions to consult the Bank prior to the first-time adoption of public cloud for critical systems, and to notify the Bank on any subsequent adoption of public cloud for critical systems. This new requirement will necessitate the organization to establish a process for consulting and notifying the Bank regarding the use of public cloud for critical systems, which may impact the organization's cloud adoption strategy and implementation timelines."
4,5,Appendix 10,"A new Appendix 10 has been added, which provides additional guidance to financial institutions for the assessment of common key risks and considerations of control measures when financial institutions adopt public cloud for critical systems. This new appendix will require the organization to review and update its cloud adoption strategy and implementation plans to ensure alignment with the guidance provided, which may involve additional risk assessments, control measures, and documentation requirements."


# **Thank you!**
<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Thank you!
</h2>